In [53]:
import pandas as pd

In [54]:
bike_realtime_df = pd.read_csv('./bike_realtime0416-0519.csv')
mrt_history_df = pd.read_csv('./mrt_history_groupby03-04.csv')
time_table_df = pd.read_csv('./time_table.csv')
youbike_mrt_distance_df = pd.read_csv('./youbike_mrt_distance.csv')
bike_station_df = pd.read_csv('./bike_station.csv')
mrt_station_info_df = pd.read_csv('./mrt_station_info.csv')
bike_realtime_df.drop(columns="create_time",inplace=True)

In [55]:
bike_realtime_df["source_time"] = pd.to_datetime(bike_realtime_df["source_time"]) 
bike_realtime_df["month"] = bike_realtime_df["source_time"].dt.month
bike_realtime_df["date"] = bike_realtime_df["source_time"].dt.date
bike_realtime_df["hour"] = bike_realtime_df["source_time"].dt.hour
bike_realtime_df["mins"] = bike_realtime_df["source_time"].dt.minute

In [56]:
bike_realtime_df["date"] = pd.to_datetime(bike_realtime_df["date"])
time_table_df["date"] = pd.to_datetime(time_table_df["date"])
mrt_history_df["date"] = pd.to_datetime(mrt_history_df["date"])

In [57]:
start_date = '2024-04-16'
end_date = '2024-04-30'

mrt_history_df = mrt_history_df[(mrt_history_df["date"] >= start_date) & (mrt_history_df["date"] <= end_date)]
bike_realtime_df = bike_realtime_df[(bike_realtime_df["date"] >= start_date) & (bike_realtime_df["date"] <= end_date)]

In [58]:
aval_bike_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_bike"].mean()).reset_index(drop=False)
aval_bike_mean.rename(columns={"aval_bike":"aval_bike_mean"},inplace=True)
aval_space_mean = pd.DataFrame(bike_realtime_df.groupby(["bike_station_id","date","hour"])["aval_space"].mean()).reset_index(drop=False)
aval_space_mean.rename(columns={"aval_space":"aval_space_mean"},inplace=True)
df_bike_hour_grby = aval_bike_mean.merge(aval_space_mean,
                     how="left",
                     left_on=["bike_station_id","date","hour"],
                     right_on=["bike_station_id","date","hour"])

In [59]:
mrt_station_table = youbike_mrt_distance_df.merge(mrt_station_info_df.loc[:,["mrt_station_id","station_name"]],
                                 on="mrt_station_id",
                                 how="left")
mrt_station_table

,bike_station_id,mrt_station_id,distance,station_name
0,500107102,O02,6.986596,景安
1,500107099,O02,7.208812,景安
2,500107065,O02,6.809924,景安
3,500107086,O02,11.338723,景安
4,500107045,O02,6.757624,景安
...,...,...,...,...
171210,500119045,O54,10.703496,蘆洲
171211,500119070,O54,11.054859,蘆洲
171212,500119048,O54,10.971054,蘆洲
171213,500119077,O54,10.791606,蘆洲


In [60]:
mrt_station_table = mrt_station_table.loc[mrt_station_table["distance"]<=0.5,]
mrt_station_table

,bike_station_id,mrt_station_id,distance,station_name
1422,500107153,BR15,0.352221,劍南路
1502,500107132,BR15,0.072653,劍南路
1509,500107133,BR15,0.242977,劍南路
1568,500107008,BR15,0.093802,劍南路
1859,500108121,BR15,0.457363,劍南路
...,...,...,...,...
166908,500113068,BL10,0.023105,龍山寺
166909,500113043,BL10,0.326785,龍山寺
166910,500113023,BL10,0.401119,龍山寺
166913,500113042,BL10,0.390115,龍山寺


In [61]:
mrt_history_df

,date,hour,mrt_station,enter_num,exit_num
113988,2024-04-16,0,龍山寺,127,142.0
113989,2024-04-16,0,麟光,10,30.0
113990,2024-04-16,0,頭前庄,15,35.0
113991,2024-04-16,0,頂溪,71,230.0
113992,2024-04-16,0,頂埔,8,104.0
...,...,...,...,...,...
151153,2024-04-30,23,三重國小,116,423.0
151154,2024-04-30,23,三重,87,183.0
151155,2024-04-30,23,三民高中,89,453.0
151156,2024-04-30,23,三和國中,99,356.0


In [62]:
mrt_cover_range_table_df = mrt_station_table.merge(mrt_history_df,left_on=["station_name"],right_on=["mrt_station"],how="left")
mrt_cover_range_table_df

,bike_station_id,mrt_station_id,distance,station_name,date,hour,mrt_station,enter_num,exit_num
0,500107153,BR15,0.352221,劍南路,2024-04-16,0.0,劍南路,79.0,31.0
1,500107153,BR15,0.352221,劍南路,2024-04-16,1.0,劍南路,0.0,0.0
2,500107153,BR15,0.352221,劍南路,2024-04-16,5.0,劍南路,0.0,0.0
3,500107153,BR15,0.352221,劍南路,2024-04-16,6.0,劍南路,103.0,190.0
4,500107153,BR15,0.352221,劍南路,2024-04-16,7.0,劍南路,561.0,862.0
...,...,...,...,...,...,...,...,...,...
301150,500113044,BL10,0.344044,龍山寺,2024-04-30,19.0,龍山寺,1418.0,2322.0
301151,500113044,BL10,0.344044,龍山寺,2024-04-30,20.0,龍山寺,1080.0,1554.0
301152,500113044,BL10,0.344044,龍山寺,2024-04-30,21.0,龍山寺,1046.0,1533.0
301153,500113044,BL10,0.344044,龍山寺,2024-04-30,22.0,龍山寺,765.0,1338.0


In [63]:
mrt_cover_range_table_df.isna().sum()

bike_station_id     0
mrt_station_id      0
distance            0
station_name        0
date               15
hour               15
mrt_station        15
enter_num          15
exit_num           15
dtype: int64

In [64]:
df1 = pd.DataFrame(mrt_cover_range_table_df.groupby(["bike_station_id","date","hour"])["enter_num"].sum()).reset_index(drop=False)
df2 = pd.DataFrame(mrt_cover_range_table_df.groupby(["bike_station_id","date","hour"])["exit_num"].sum()).reset_index(drop=False)
df3 = pd.DataFrame(mrt_cover_range_table_df.groupby(["bike_station_id","date","hour"])["mrt_station"].nunique()).reset_index(drop=False)

In [65]:
df = df1.merge(
    df2,
    left_on=["bike_station_id","date","hour"],
    right_on=["bike_station_id","date","hour"],
    how = "left"
)
df = df.merge(
    df3,
    left_on=["bike_station_id","date","hour"],
    right_on=["bike_station_id","date","hour"],
    how = "left"
)
df

,bike_station_id,date,hour,enter_num,exit_num,mrt_station
0,500101001,2024-04-16,0.0,60.0,53.0,1
1,500101001,2024-04-16,1.0,0.0,0.0,1
2,500101001,2024-04-16,5.0,0.0,0.0,1
3,500101001,2024-04-16,6.0,152.0,172.0,1
4,500101001,2024-04-16,7.0,768.0,949.0,1
...,...,...,...,...,...,...
224905,500119082,2024-04-30,19.0,2213.0,1827.0,1
224906,500119082,2024-04-30,20.0,1642.0,1143.0,1
224907,500119082,2024-04-30,21.0,1672.0,1050.0,1
224908,500119082,2024-04-30,22.0,1598.0,833.0,1


In [66]:
bike_station_df

,bike_station_id,station_name,total_space,lat,lng,district,address,disable,create_time,source_time
0,500107008,YouBike2.0_捷運劍南路站(2號出口),86,25.08401,121.55535,中山區,敬業三路11號(植福路側對面),1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00
1,500107028,YouBike2.0_捷運松江南京站(7號出口),75,25.05246,121.53320,中山區,松江路119號前,1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00
2,500107035,YouBike2.0_捷運中山國小站(2號出口),59,25.06246,121.52705,中山區,捷運中山國小站(2號出口)後方,1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00
3,500106067,YouBike2.0_臺北轉運站,68,25.04842,121.51994,中正區,市民大道一段/中山北路一段(西北側),1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00
4,500106014,YouBike2.0_牯嶺公園,58,25.02338,121.51889,中正區,廈門街113巷/牯嶺街口(西側),1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00
...,...,...,...,...,...,...,...,...,...,...
1412,500109007,YouBike2.0_關渡宮,62,25.11739,121.46288,北投區,知行路360號(旁),1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00
1413,500104108,YouBike2.0_捷運芝山站(2號出口)_1,62,25.10336,121.52263,士林區,福華路/福華路162巷(東南側),1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00
1414,500103028,YouBike2.0_捷運北門站(3號出口),62,25.04991,121.51040,大同區,塔城街與鄭州路口西南側,1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00
1415,500105018,YouBike2.0_國立政治大學,62,24.98737,121.57726,文山區,指南路二段153號(對面),1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00


In [67]:
bike_completed_df = df_bike_hour_grby.merge(bike_station_df.loc[:,["bike_station_id","total_space",	"lat",	"lng"]],
                                            on="bike_station_id",
                                           how="left"   
                                           )

In [68]:
bike_completed_df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,total_space,lat,lng
0,500101001,2024-04-16,22,7.833333,20.166667,28,25.02605,121.54360
1,500101001,2024-04-16,23,23.000000,5.000000,28,25.02605,121.54360
2,500101001,2024-04-17,0,20.500000,7.500000,28,25.02605,121.54360
3,500101001,2024-04-17,1,16.666667,11.333333,28,25.02605,121.54360
4,500101001,2024-04-17,2,18.000000,10.000000,28,25.02605,121.54360
...,...,...,...,...,...,...,...,...
438546,500119091,2024-04-29,16,11.500000,6.500000,18,25.01816,121.54469
438547,500119091,2024-04-29,17,9.000000,9.000000,18,25.01816,121.54469
438548,500119091,2024-04-29,18,0.500000,17.500000,18,25.01816,121.54469
438549,500119091,2024-04-29,19,6.666667,11.333333,18,25.01816,121.54469


In [69]:
bike_completed_df.dtypes

bike_station_id             int64
date               datetime64[ns]
hour                        int32
aval_bike_mean            float64
aval_space_mean           float64
total_space                 int64
lat                       float64
lng                       float64
dtype: object

In [70]:
df["hour"] = df["hour"].astype(int)

In [71]:
df_res = df.merge(bike_completed_df,
         left_on=["bike_station_id","date","hour"],
         right_on=["bike_station_id","date","hour"],
         how="left")

In [72]:
df_res = df_res.loc[~(df_res["aval_bike_mean"].isna()),]

In [73]:
df_res = df_res.loc[~df["hour"].isin([22,23,0,1,2,3,4,5]),]

In [74]:
df_res["bike_rate"] = df_res["aval_bike_mean"]/df_res["total_space"]
df_res["total_visitors"] = df_res["enter_num"] + df_res["exit_num"]
df_res["ex_enter_diff"] =  df_res["exit_num"] - df_res["enter_num"] 
df_res["enter_ex_diff"] =  df_res["enter_num"] - df_res["exit_num"]  

In [75]:
df_res.sort_values(by=["bike_station_id","date","hour"])

,bike_station_id,date,hour,enter_num,exit_num,mrt_station,aval_bike_mean,aval_space_mean,total_space,lat,lng,bike_rate,total_visitors,ex_enter_diff,enter_ex_diff
24,500101001,2024-04-17,6,152.0,193.0,1,23.500000,4.500000,28.0,25.02605,121.54360,0.839286,345.0,41.0,-41.0
25,500101001,2024-04-17,7,741.0,938.0,1,17.833333,10.166667,28.0,25.02605,121.54360,0.636905,1679.0,197.0,-197.0
26,500101001,2024-04-17,8,1774.0,2219.0,1,3.166667,24.833333,28.0,25.02605,121.54360,0.113095,3993.0,445.0,-445.0
27,500101001,2024-04-17,9,982.0,1402.0,1,0.666667,27.333333,28.0,25.02605,121.54360,0.023810,2384.0,420.0,-420.0
28,500101001,2024-04-17,10,470.0,678.0,1,0.500000,27.500000,28.0,25.02605,121.54360,0.017857,1148.0,208.0,-208.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224881,500119082,2024-04-29,16,1747.0,1286.0,1,7.000000,3.000000,10.0,25.01703,121.53781,0.700000,3033.0,-461.0,461.0
224882,500119082,2024-04-29,17,2994.0,2172.0,1,0.000000,10.000000,10.0,25.01703,121.53781,0.000000,5166.0,-822.0,822.0
224883,500119082,2024-04-29,18,3326.0,2169.0,1,0.000000,10.000000,10.0,25.01703,121.53781,0.000000,5495.0,-1157.0,1157.0
224884,500119082,2024-04-29,19,2234.0,1660.0,1,0.000000,10.000000,10.0,25.01703,121.53781,0.000000,3894.0,-574.0,574.0


In [76]:
bk_ids = list(df_res["bike_station_id"].unique())
corr_df = pd.DataFrame(columns=["bike_station_id","corr_enter_bike","corr_exit_bike","corr_total_visitors","corr_ex_enter_diff","corr_enter_exit"])

In [77]:
for bk_id in bk_ids:
    tem = df_res.loc[df_res["bike_station_id"]==bk_id,]
    cor_matrix = tem.loc[:,["bike_rate","enter_num","total_visitors","ex_enter_diff","enter_ex_diff","exit_num"]].corr()
    corr_df_tem = pd.DataFrame({
        "bike_station_id":[bk_id],
        "corr_enter_bike":[cor_matrix.loc["enter_num","bike_rate"]],
        "corr_exit_bike":cor_matrix.loc["exit_num","bike_rate"],
        "corr_total_visitors":cor_matrix.loc["total_visitors","bike_rate"],
        "corr_ex_enter_diff":cor_matrix.loc["ex_enter_diff","bike_rate"],
        "corr_enter_ex_diff":cor_matrix.loc["enter_ex_diff","bike_rate"],
        "corr_enter_exit":cor_matrix.loc["exit_num","enter_num"]
    })
    corr_df = pd.concat([corr_df,corr_df_tem],axis=0)

C:\Users\andy\AppData\Local\Temp\ipykernel_19844\2072460919.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  corr_df = pd.concat([corr_df,corr_df_tem],axis=0)


In [79]:
corr_df.sort_values(["corr_ex_enter_diff"],ascending=True,inplace=True)
corr_df

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit,corr_enter_ex_diff
0,500119048,0.557353,-0.031539,0.351141,-0.702179,0.578933,0.702179
0,500113040,0.106305,-0.483300,-0.243294,-0.673540,0.583025,0.673540
0,500105005,0.355522,-0.626543,-0.144482,-0.663768,-0.046347,0.663768
0,500101181,0.466719,-0.063419,0.276839,-0.619741,0.578933,0.619741
0,500110076,0.554818,-0.163543,0.286596,-0.613494,0.256228,0.613494
...,...,...,...,...,...,...,...
0,500105062,NaN,NaN,NaN,NaN,0.069693,NaN
0,500109065,NaN,NaN,NaN,NaN,0.828852,NaN
0,500110028,NaN,NaN,NaN,NaN,0.034303,NaN
0,500111085,NaN,NaN,NaN,NaN,0.665863,NaN


In [85]:
bike_corr_high_id = corr_df.loc[corr_df["corr_enter_ex_diff"]>=0.5,"bike_station_id"]

In [87]:
df.loc[df["bike_station_id"].isin(bike_corr_high_id),]

,bike_station_id,date,hour,enter_num,exit_num,mrt_station
4095,500101021,2024-04-16,0,215.0,110.0,1
4096,500101021,2024-04-16,1,5.0,0.0,1
4097,500101021,2024-04-16,5,0.0,0.0,1
4098,500101021,2024-04-16,6,306.0,381.0,1
4099,500101021,2024-04-16,7,987.0,1936.0,1
...,...,...,...,...,...,...
220180,500119048,2024-04-30,19,2213.0,1827.0,1
220181,500119048,2024-04-30,20,1642.0,1143.0,1
220182,500119048,2024-04-30,21,1672.0,1050.0,1
220183,500119048,2024-04-30,22,1598.0,833.0,1


In [82]:
bike_station_df.loc[bike_station_df["bike_station_id"]==500105005,]

,bike_station_id,station_name,total_space,lat,lng,district,address,disable,create_time,source_time
164,500105005,YouBike2.0_捷運萬隆站(2號出口),12,25.00144,121.53983,文山區,興隆路一段70巷/羅斯福路5段211巷26弄口,1,2024-05-19 23:32:27.051988+00:00,2024-05-19 23:28:23+00:00


In [24]:
df_res.to_csv("test3.csv",index=False,encoding="utf-8-sig")